# PCA on roles

In [1]:
import os
import sys
import torch
import numpy as np
import pandas as pd

sys.path.append('.')
sys.path.append('..')

from utils.pca_utils import *
from plots import *

## Configuration

In [39]:
# Configuration - Change these parameters for different models/datasets
base_dir = "/workspace/gemma-2-27b"
type = "roles_240"
dir = f"{base_dir}/{type}"
model_name = "Gemma 2 27B"
layer = 34

In [29]:
# 30 or 240
if type == "roles":
    n_questions = 30
    n_prompt_types = 2
elif type == "roles_240":  # roles_240 or other patterns
    n_questions = 240
    n_prompt_types = 1

## Load vectors

In [30]:
# load all vectors 
vector_dir = f"{dir}/vectors"

# iterate through each .pt file in the directory
vectors = {}
for file in os.listdir(vector_dir):
    if file.endswith(".pt"):
        vectors[file.replace(".pt", "")] = torch.load(os.path.join(vector_dir, file))

print(f"Found {len(vectors.keys())} roles with vectors")

Found 275 roles with vectors


In [22]:
# load default vectors
default_vectors = torch.load(f"{dir}/default_vectors.pt")

In [23]:
print(vectors['graduate'].keys())
print(default_vectors.keys())
print(default_vectors['activations'].keys())

dict_keys(['pos_2', 'pos_3', 'pos_all'])
dict_keys(['activations', 'metadata'])
dict_keys(['pos_1', 'all_1', 'default_1'])


## PCA 

In [24]:
pos_2_roles = []
pos_2_vectors = []
pos_3_roles = []
pos_3_vectors = []

# get the vectors keys for pos_2 and pos_3 for each role
for role, vector in vectors.items():
    if 'pos_2' in vector.keys():
        pos_2_roles.append(role)
        pos_2_vectors.append(vector['pos_2'])
    if 'pos_3' in vector.keys():
        pos_3_roles.append(role)
        pos_3_vectors.append(vector['pos_3'])

print(len(pos_2_roles))
print(len(pos_3_roles))

combined_vectors = pos_2_vectors + pos_3_vectors

173
275


In [25]:
float_stack_vectors = torch.stack(combined_vectors).float()
print(float_stack_vectors.shape)

torch.Size([448, 46, 4608])


In [40]:
pca_transformed, variance_explained, n_components, pca, scaler = compute_pca(float_stack_vectors, layer)

PCA fitted with 448 components
Cumulative variance for first 5 components: [0.16902802 0.25303985 0.30386488 0.35273554 0.38629897]

PCA Analysis Results:
Elbow point at component: 2
Dimensions for 70% variance: 28
Dimensions for 80% variance: 49
Dimensions for 90% variance: 90
Dimensions for 95% variance: 133


In [41]:
results = {}
results['layer'] = layer
results['roles'] = {
    'pos_2': pos_2_roles,
    'pos_3': pos_3_roles
}
results['vectors'] = {
    'pos_2': pos_2_vectors,
    'pos_3': pos_3_vectors
}
results['pca_transformed'] = pca_transformed
results['variance_explained'] = variance_explained
results['n_components'] = n_components
results['pca'] = pca
results['scaler'] = scaler

pca_dir = f"{dir}/pca"
os.makedirs(pca_dir, exist_ok=True)
torch.save(results, f"{pca_dir}/layer{layer}_pos23.pt")
print(f"Saved PCA results to {pca_dir}/layer{layer}_pos23.pt")

Saved PCA results to /workspace/gemma-2-27b/roles_240/pca/layer34_pos23.pt


In [ ]:
# also compute for normalized vectors
normalized_vectors = F.normalize(float_stack_vectors, p=2, dim=-1)

norms = normalized_vectors.norm(p=2, dim=-1)
print(norms.mean().item(), norms.std().item())

In [ ]:
pca_transformed, variance_explained, n_components, pca, scaler = compute_pca(normalized_vectors, layer, scaler=False)

In [ ]:
results = {}
results['layer'] = layer
results['roles'] = {
    'pos_2': pos_2_roles,
    'pos_3': pos_3_roles
}
results['vectors'] = {
    'pos_2': pos_2_vectors,
    'pos_3': pos_3_vectors
}
results['pca_transformed'] = pca_transformed
results['variance_explained'] = variance_explained
results['n_components'] = n_components
results['pca'] = pca
results['scaler'] = scaler

pca_dir = f"{dir}/pca"
os.makedirs(pca_dir, exist_ok=True)
torch.save(results, f"{pca_dir}/layer{layer}_normalized_pos23.pt")

## Plots

In [33]:
type = "pos23"

plot_dir = f"/root/git/plots/results/{model_name.lower().replace(' ', '-')}/roles"
os.makedirs(plot_dir, exist_ok=True)

normalized = False
if normalized:
    pca_results = torch.load(f"{dir}/pca/layer{layer}_normalized_pos23.pt", weights_only=False)
else:
    pca_results = torch.load(f"{dir}/pca/layer{layer}_pos23.pt", weights_only=False)

In [18]:
# flip PC1 for llama
pca_results['pca'].components_[0] *= -1
pca_results['pca_transformed'][:, 0] *= -1
torch.save(pca_results, f"{dir}/pca/layer{layer}_pos23.pt")

In [34]:
# load in PCs
default_vectors = torch.load(f"{dir}/default_vectors.pt")

In [35]:
# also calculate role labels for plotting
def get_role_labels(pca_results):
    labels = []
    if 'pos_2' in pca_results['roles'].keys():
        pos_2_roles = [role.replace('_', ' ').title() for role in pca_results['roles']['pos_2']]
        pos_2_roles = [f"{role} (Somewhat RP)" for role in pos_2_roles]
        labels.extend(pos_2_roles)
    if 'pos_3' in pca_results['roles'].keys():
        pos_3_roles = [role.replace('_', ' ').title() for role in pca_results['roles']['pos_3']]
        pos_3_roles = [f"{role} (Fully RP)" for role in pos_3_roles]
        labels.extend(pos_3_roles)
    return labels

role_labels = get_role_labels(pca_results)



In [36]:
print(role_labels[:10])
print(role_labels[-10:])

['Writer (Somewhat RP)', 'Workaholic (Somewhat RP)', 'Witness (Somewhat RP)', 'Visionary (Somewhat RP)', 'Virus (Somewhat RP)', 'Virtuoso (Somewhat RP)', 'Vigilante (Somewhat RP)', 'Veterinarian (Somewhat RP)', 'Vegan (Somewhat RP)', 'Validator (Somewhat RP)']
['Altruist (Fully RP)', 'Alien (Fully RP)', 'Advocate (Fully RP)', 'Adolescent (Fully RP)', 'Addict (Fully RP)', 'Actor (Fully RP)', 'Activist (Fully RP)', 'Accountant (Fully RP)', 'Absurdist (Fully RP)', 'Aberration (Fully RP)']


In [37]:
# get default activation and project into PCA space
assistant_layer_activation = default_vectors['activations']['default_1'][layer, :].float().reshape(1, -1)
if not normalized:
    asst_scaled = pca_results['scaler'].transform(assistant_layer_activation)
    asst_projected = pca_results['pca'].transform(asst_scaled)
else:
    asst_normalized = F.normalize(assistant_layer_activation, p=2, dim=-1)
    asst_projected = pca_results['pca'].transform(asst_normalized)

assistant_layer_activation = assistant_layer_activation.numpy()


In [38]:
if normalized:
    subtitle = f"{model_name.replace('-', ' ')}, Layer {layer} - Shared Question Set, Unit Normalized Vectors"
else:
    subtitle = f"{model_name.replace('-', ' ')}, Layer {layer} - Shared Question Set, Mean-Centered and Scaled Vectors"

for i in range(1):
    fig = plot_pc(
        pca_results=pca_results,
        role_labels=role_labels,
        layer=layer,
        pc_component=i,
        assistant_activation=default_vectors['activations']['default_1'],
        assistant_projection=asst_projected[0],
        title="PCA on Role-Playing Vectors",
        subtitle=subtitle,
        scaled=not normalized,
    )
    fig.show()

    if not normalized:
        fig.write_html(f"{plot_dir}/pc{i+1}.html")
    else:
        fig.write_html(f"{plot_dir}/pc{i+1}_normalized.html")


In [ ]:
fig_3d = plot_3d_pca(
    pca_results,
    role_labels,
    "pos23",
    assistant_projection=asst_projected[0],
    subtitle=f"{model_name.replace('-', ' ')}, Layer {layer} - Shared Question Set"
)
fig_3d.show()
#fig_3d.write_html(f"{plot_dir}/pca_3d.html")